In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[1]").appName("sql_read").getOrCreate()

In [3]:
user = "praveen"
password = "pr@123.com"
database = "praveen_db"
table = "dbo.family"
#PRAVEEN
#localhost/1433

In [5]:
jdbcDF.show()

+---------+------+---+---------+
|member_id|  name|age|parent_id|
+---------+------+---+---------+
|        1|  jhon| 70|     null|
|        2|   set| 40|        1|
|        3| joset| 35|        1|
|        4|  sony| 18|        2|
|        5|mondey| 12|        2|
|        6|  jhon|  8|        3|
+---------+------+---+---------+



In [7]:
jdbcDF = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlserver://localhost:1433;encrypt=true;trustServerCertificate=true;databaseName={praveen_db};") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("user", user) \
    .option("password", password) \
    .option("query",'select * from family where age>15')\
    .load()

In [8]:
jdbcDF.show()

+---------+-----+---+---------+
|member_id| name|age|parent_id|
+---------+-----+---+---------+
|        1| jhon| 70|     null|
|        2|  set| 40|        1|
|        3|joset| 35|        1|
|        4| sony| 18|        2|
+---------+-----+---+---------+



In [7]:
rdd1 = jdbcDF.rdd

In [11]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer
def _to_java_object_rdd(rdd):  
    """ Return a JavaRDD of Object by unpickling
    It will convert each Python object into Java object by Pyrolite, whenever the
    RDD is serialized in batch or not.
    """
    rdd = rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
    return rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)

JavaObj = _to_java_object_rdd(rdd1)

nbytes = spark._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)

In [14]:
(nbytes/1024)/1024

3.021484375

In [15]:
spark

In [16]:
jdbcDF.cache()

DataFrame[member_id: int, name: string, age: int, parent_id: int]

In [17]:
jdbcDF.show()

+---------+------+---+---------+
|member_id|  name|age|parent_id|
+---------+------+---+---------+
|        1|  jhon| 70|     null|
|        2|   set| 40|        1|
|        3| joset| 35|        1|
|        4|  sony| 18|        2|
|        5|mondey| 12|        2|
|        6|  jhon|  8|        3|
+---------+------+---+---------+



In [20]:
rdd1.persist()

MapPartitionsRDD[7] at javaToPython at DirectMethodHandleAccessor.java:104

In [21]:
rdd1.collect()

[Row(member_id=1, name='jhon', age=70, parent_id=None),
 Row(member_id=2, name='set', age=40, parent_id=1),
 Row(member_id=3, name='joset', age=35, parent_id=1),
 Row(member_id=4, name='sony', age=18, parent_id=2),
 Row(member_id=5, name='mondey', age=12, parent_id=2),
 Row(member_id=6, name='jhon', age=8, parent_id=3)]

In [22]:
jdbcDF.unpersist()

DataFrame[member_id: int, name: string, age: int, parent_id: int]

In [23]:
rdd1.unpersist()

MapPartitionsRDD[7] at javaToPython at DirectMethodHandleAccessor.java:104